In [2]:
%load_ext autoreload
%autoreload 2

# Compute Performance Data and Save for Plotting P-Charts

In [1]:
%load_ext autoreload
%autoreload 2

import os
from performance import compute_and_save_performance
suffix = "_true.csv"
for entry in os.listdir("evaluation_data"):
    if suffix in entry:
        name = entry[:-len(suffix)]
        print(name)
        compute_and_save_performance(name)

characteristic_bolete.h5
Process ForkPoolWorker-45:
Process ForkPoolWorker-18:
Process ForkPoolWorker-20:
Process ForkPoolWorker-30:
Process ForkPoolWorker-15:
Process ForkPoolWorker-43:
Process ForkPoolWorker-38:
Process ForkPoolWorker-36:
Process ForkPoolWorker-28:
Process ForkPoolWorker-9:
Process ForkPoolWorker-32:
Process ForkPoolWorker-21:
Process ForkPoolWorker-26:
Process ForkPoolWorker-33:
Process ForkPoolWorker-31:
Process ForkPoolWorker-39:
Process ForkPoolWorker-2:
Process ForkPoolWorker-22:
Process ForkPoolWorker-44:
Process ForkPoolWorker-34:
Process ForkPoolWorker-35:
Process ForkPoolWorker-46:
Process ForkPoolWorker-23:
Process ForkPoolWorker-47:
Process ForkPoolWorker-40:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:


KeyboardInterrupt: 

# Get baselines for p-charts
## Random Performance

In [1]:
%load_ext autoreload
%autoreload 2

from performance_concurrent import *

random_char_performance(5)

=====================================================================================================Process ForkPoolWorker-7:
Process ForkPoolWorker-35:
Process ForkPoolWorker-22:
Process ForkPoolWorker-47:
Process ForkPoolWorker-43:
Process ForkPoolWorker-2:
Process ForkPoolWorker-48:
Process ForkPoolWorker-6:
Process ForkPoolWorker-16:
Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-38:
Process ForkPoolWorker-36:
Process ForkPoolWorker-1:
Process ForkPoolWorker-40:
Process ForkPoolWorker-14:
Process ForkPoolWorker-31:
Process ForkPoolWorker-27:
Process ForkPoolWorker-21:
Process ForkPoolWorker-5:
Process ForkPoolWorker-42:
Process ForkPoolWorker-46:
Process ForkPoolWorker-37:
Process ForkPoolWorker-17:
Process ForkPoolWorker-15:
Process ForkPoolWorker-45:
Process ForkPoolWorker-23:
Process ForkPoolWorker-28:
Process ForkPoolWorker-18:
Process ForkPoolWorker-39:
Process ForkPoolWorker-32:
Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Process ForkP

KeyboardInterrupt: 

## Perfect Performance

In [4]:
import numpy as np
from data_utils import save_performance_data
from lookalikes import get_feats, get_num_species, get_performance

density = 100
C = 38
results = np.zeros((density,2))
p_num = 0
for p in range(101):
    print("=", end="")

    perf = np.zeros((get_num_species(),))
    
    for ids in range(get_num_species()):
        u = get_feats(ids)
        perf[ids] = get_performance(u, ids, p/100)
    
    results[p_num,:] = [p/100, np.mean(perf)]

save_performance_data(results[:,0], results[:,1], "perfect")

True

## Compare tau hats

In [48]:
from performance import * 
from lookalikes import *
from data_utils import load_raw_eval_data
import os


def shared(tau, tau_hat):
    return len(np.intersect1d(tau, tau_hat))

suffix = "_true.csv"
entries = os.listdir("evaluation_data")

modeltypes = ["characteristic", "direct"]
filetypes = ["_bolete.h5", "_bolete_lowres.h5"]
ps = {filetype:{} for filetype in filetypes}
for p in range(100):
    p = p/100
    tau_hats = {}
    for modeltype in modeltypes:
        for filetype in filetypes:
            name = modeltype+filetype
            scores, y_pred, y_true, y_labels = load_raw_eval_data(name)
            tau_hats[name] = []
            for i in range(len(y_labels)):
                if modeltype == "direct":
                    i_scores = list(enumerate(scores[i]))
                    sorted_scores = sorted(i_scores, key=lambda x: x[1], reverse=True)
                    size_of_tau_hat = len(lookalikes(sorted_scores[0][0], p))
                    tau_hat = [idx for (idx, val) in sorted_scores[:size_of_tau_hat]]
                else:
                    tau_hat = species_from_feats(y_pred[i], p)
                tau_hats[name].append(tau_hat)
    
    for filetype in filetypes:
        tau_hat_char = tau_hats["characteristic"+filetype]
        tau_hat_dir = tau_hats["direct"+filetype]
        num_examples = len(tau_hat_char)

        percent_shared = 0
        print("=", end='')
        for i in range(num_examples):
            total_unique = set(list(tau_hat_char[i]) + list(tau_hat_dir[i]))
            total = len(total_unique)
            num_shared = shared(tau_hat_char[i], tau_hat_dir[i])
            percent_shared += num_shared/total
        percent_shared /= num_examples
        ps[filetype][p] = round(percent_shared,2)

for filetype in filetypes:
    p_vals = [val for val in ps[filetype].keys()]
    sims = [sim for sim in ps[filetype].values()]
    # print('p vals = ', p_vals)
    # print('sims = ', sims)
    np.savetxt("performance_data/" + filetype[1:] + "_tau_hat_similarity.csv", [p_vals, sims], delimiter=",", fmt="%f",)

========================================================================================================================================================================================================p vals =  [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
sims =  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.98, 0.98, 0.98, 0.98, 0.97, 0.97, 0.96, 0.95, 0.95, 0.93, 0.93, 0.91, 0.9, 0.9, 0